In [15]:
import numpy as np
from collections import defaultdict
import gtnlplib.preproc
import gtnlplib.constants
import scipy.io
import gtnlplib.wordlist

In [2]:
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TRAINKEY)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.DEVKEY)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TESTKEY)

In [35]:

ac_train = gtnlplib.preproc.getAllCounts(gtnlplib.preproc.dataIterator(gtnlplib.constants.TRAINKEY, test_mode=False))
trainData = gtnlplib.preproc.dataIterator(gtnlplib.constants.TRAINKEY, test_mode=False)
devData = gtnlplib.preproc.dataIterator(gtnlplib.constants.DEVKEY, test_mode=False)
testData = gtnlplib.preproc.dataIterator(gtnlplib.constants.TESTKEY, test_mode=True)


In [27]:

word_idx = {}
for w in ac_train:
    word_idx[w] = len(word_idx)
poswords, negwords = gtnlplib.wordlist.loadSentimentWords (gtnlplib.constants.SENTIMENT_FILE)


In [37]:
def get_XY(keyfile, testmode):
    idx_dict = {'POS' : 0, 'NEG' : 1, 'NEU' : 2, 'UNK' : 3}
    X = []
    Y = []
    data_iter = gtnlplib.preproc.dataIterator(keyfile, test_mode=testmode)
    for i,(feat,label) in enumerate(data_iter):
        Y.append(idx_dict[label])
        cur = np.zeros(len(word_idx) + 2)
        for w in feat:
            if w in word_idx:
                cur[word_idx[w]] = feat[w]        
            if w in poswords:
                cur[len(word_idx)] += feat[w]
            elif w in negwords:
                cur[len(word_idx) + 1] += feat[w]
        X.append(cur)
    Y = np.array(Y)
    X = np.array(X)
    return X, Y
train_X, train_Y = get_XY(gtnlplib.constants.TRAINKEY, False)
dev_X, dev_Y = get_XY(gtnlplib.constants.DEVKEY, False)
test_X, test_Y = get_XY(gtnlplib.constants.TESTKEY, True)
output_mat = 'bow-train_dev.mat'
result = {'trainx' : train_X, 'trainy' : train_Y, 'devx' : dev_X, 'devy' : dev_Y, 'testx' : test_X}
scipy.io.savemat(output_mat, result)